# Tutorial for Cosmos DB and Python for the SQL API

As you probably know, [Cosmos DB](https://docs.microsoft.com/en-us/azure/cosmos-db/) is a globally distributed, multi-model database service in Azure that supports document, key-value, wide-column, and graph databases.

Cosmos DB includes support for multiple Software Development Kits for programming languages such as Python. Multiple resources already exist that help to learn to use the Python SDK: 

* [Cosmos DB reference guide for the Python SDK](https://docs.microsoft.com/en-us/python/api/azure-cosmos/?view=azure-python)
* [Cosmos DB samples with the Python SDK](https://github.com/Azure/azure-cosmos-python/blob/master/test/crud_tests.py)

As opposed to documentation or code samples, Jupyter Notebooks offer a very interesting combination of rich text for explanations and pieces of code that can be executed independently from each other, which makes it ideal to learn a new technology.

This notebook will go over the basic (and not so basic) concepts of using Cosmos DB with its Python SDK for the SQL API. You can run this repository from any system with Python 3.x, or using the free platform [Azure Notebooks](https://notebooks.azure.com/), where you have the option of importing Github repositories.

## Creating your Cosmos DB account

All Cosmos databases are contained in a Cosmos DB account. You can create a Cosmos DB account in Azure using any Azure mechanism. In this notebook we will assume that you have installed the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest), and use it to create a new resource group and a Cosmos DB account. But first things first, you need to login to Azure, and select the subscription you want to use"

In [7]:
!az login

[
  {
    "cloudName": "AzureCloud",
    "id": "e7da9914-9b05-4891-893c-546cb7b0422e",
    "isDefault": true,
    "name": "Azure CXP FTA Internal Subscription JOMORE",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "jomore@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "3e78e84b-6750-44b9-9d57-d9bba935237a",
    "isDefault": false,
    "name": "MSDN JOMORE",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "jomore@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "333e402a-65a0-45a9-8e23-867ca146c290",
    "isDefault": false,
    "name": "Edge-ES-CI-Manual",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "jomore@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "id": "64e355d7-997c-49

In [ ]:
!az account set -s "your subscription name"

We will now set some variables containing the name of the resource group we will use, the name of the Cosmos DB account to be created, the name for a database and for a collection. Consider that the name for your CosmosDB must be globally unique:

In [3]:
rg         = "cosmoslab"
location   = "westeurope"

account    = "mycosmos1138"
db_name    = "ecommerce"
coll_name  = "customers"

And now we can create our resource group and Cosmos DB account. When creating the account there some attributes you can defined, such as the type of CosmosDB and API (in this case we will use the SQL API aka DocumentDB), the locations (in this example we will use only one region).

Creating the account will take some minutes, please be patient.

In [4]:
!az group create -n "$rg" -l "$location"
!az cosmosdb create -g "$rg" -n "$account" --locations "$location"=0 --kind GlobalDocumentDB

{
  "id": "/subscriptions/e7da9914-9b05-4891-893c-546cb7b0422e/resourceGroups/cosmoslab",
  "location": "westeurope",
  "managedBy": null,
  "name": "cosmoslab",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}
{
  "capabilities": [],
  "consistencyPolicy": {
    "defaultConsistencyLevel": "Session",
    "maxIntervalInSeconds": 5,
    "maxStalenessPrefix": 100
  },
  "databaseAccountOfferType": "Standard",
  "documentEndpoint": "https://mycosmos1138.documents.azure.com:443/",
  "enableAutomaticFailover": false,
  "failoverPolicies": [
    {
      "failoverPriority": 0,
      "id": "mycosmos1138-westeurope",
      "locationName": "West Europe"
    }
  ],
  "id": "/subscriptions/e7da9914-9b05-4891-893c-546cb7b0422e/resourceGroups/cosmoslab/providers/Microsoft.DocumentDB/databaseAccounts/mycosmos1138",
  "ipRangeFilter": "",
  "isVirtualNetworkFilterEnabled": false,
  "kind": "GlobalDocumentDB",
  "location": "West Europe",
  "name": "mycosmos1138",
  "provisio

## Modify Cosmos DB account

After creating an account, you can modify it, for example to add new read or write regions. In this example we will add a second read region with a failover priority of 1:

In [5]:
!az cosmosdb update -g "$rg" -n "$account" --locations "$location"=0 northeurope=1

{
  "capabilities": [],
  "consistencyPolicy": {
    "defaultConsistencyLevel": "Session",
    "maxIntervalInSeconds": 5,
    "maxStalenessPrefix": 100
  },
  "databaseAccountOfferType": "Standard",
  "documentEndpoint": "https://mycosmos1138.documents.azure.com:443/",
  "enableAutomaticFailover": false,
  "failoverPolicies": [
    {
      "failoverPriority": 0,
      "id": "mycosmos1138-westeurope",
      "locationName": "West Europe"
    },
    {
      "failoverPriority": 1,
      "id": "mycosmos1138-northeurope",
      "locationName": "North Europe"
    }
  ],
  "id": "/subscriptions/e7da9914-9b05-4891-893c-546cb7b0422e/resourceGroups/cosmoslab/providers/Microsoft.DocumentDB/databaseAccounts/mycosmos1138",
  "ipRangeFilter": "",
  "isVirtualNetworkFilterEnabled": false,
  "kind": "GlobalDocumentDB",
  "location": "West Europe",
  "name": "mycosmos1138",
  "provisioningState": "Succeeded",
  "readLocations": [
    {
      "documentEndpoint": "https://mycosmos1138-westeurope.documents

## Authentication

Authentication to a Cosmos DB account works using an account key. There are two account keys for key rotation. You can use the Azure CLI to get the key for an existing account, in this case we get the primary key:

In [19]:
account_key = !az cosmosdb list-keys -g $rg -n $account --query primaryMasterKey -o tsv
# Per default the output is a list, keep only the first item
account_key = str(account_key[0])

## Initialize the client

You can create databases and collections using the Azure CLI too, or using the Python SDK already. Since this notebook is about Python, let us use the Python SDK already for this.

The first thing we need is to initialize the client, with the Cosmos DB account endpoint (that you can derive from the name, or use the Azure CLI to find out), and the master key that we already got.

In [21]:
# Option 1: Building the endpoint as a string
cosmosdb_endpoint = "https://{0}.documents.azure.com".format(account)
print(cosmosdb_endpoint)

https://mycosmos1138.documents.azure.com


In [25]:
# Option 2: Getting the endpoint with the Azure CLI
cosmosdb_endpoint = ! az cosmosdb show -g $rg -n $account --query documentEndpoint -o tsv
cosmosdb_endpoint = str(cosmosdb_endpoint[0])
print(cosmosdb_endpoint)

https://mycosmos1138.documents.azure.com:443/


Before initializaing the client, let us make sure that we have all required Python dependencies:

In [6]:
!pip install -r requirements.txt

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [26]:
import azure.cosmos.cosmos_client as cosmos_client

client = cosmos_client.CosmosClient(url_connection=cosmosdb_endpoint, auth={'masterKey': account_key})

## Create Database and Collection

With an initialized client we can now create a database and a collection. Note that you can allocate a performance to the database or to the collection. In this example we will allocate the minimum allowed performance (400 RU/s) to the collection.

See the reference guide for the [createDatabase method](https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python#createdatabase-database--options-none-) and for the [createContainer method](https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python#createcontainer-database-link--collection--options-none-).

Note how when creating a container (called "collection" for document-based schemas) you specify some attributes as part as the collection definition (such as the partition key or the indexing), and others as options (such as the throughput).

In [27]:
database_definition = { 'id': db_name }
created_db = client.CreateDatabase(database_definition)
db_link=created_db['_self']

In [28]:
# Before creating the collection, this code makes sure it does not exist before
# Otherwise it gives out an error, that you could grab in a try/catch block as well
coll_query = "select * from r where r.id = '{0}'".format(coll_name)
coll = list(client.QueryContainers(db_link, coll_query))
if coll:
    print("Collection", coll_name, "already exists")
else:
    collection_definition = {
        'id': coll_name, 
        'indexingPolicy': {'indexingMode': 'consistent'},
        'partitionKey': {
            'paths': [
              '/id'
            ],
            'kind': 'Hash'
        }
    }
    collection_options = { 'offerThroughput': 400 }
    created_collection = client.CreateContainer(created_db['_self'], collection_definition, collection_options)
    print("Collection", coll_name, "has been created")

Collection customers has been created


## Get an existing database or collection

If you are working with an existing database or collection, you just need to retrieve them. The retrieved object will have an s_id field that you can use to verify whether the database and collections where found or not.

In [29]:
# Get the DB link
db_query = "select * from r where r.id = '{0}'".format(db_name)
db = list(client.QueryDatabases(db_query))
if db:
    db_link = db[0]['_self']
    print("Database", db_name, "found")
    # Uncomment the next line to see the full object
    # print(db[0])

    # If the database has been found, try to find the collection
    coll_query = "select * from r where r.id = '{0}'".format(coll_name)
    coll = list(client.QueryContainers(db_link, coll_query))
    if coll:
        coll_link = coll[0]['_self']
        print("Collection", coll_name, "found:")
        print(" - Indexing policy - Mode:", coll[0]['indexingPolicy']['indexingMode'])
        print(" - Indexing policy - Mode:", coll[0]['indexingPolicy']['automatic'])
        print(" - Conflict resolution policy mode:", coll[0]['conflictResolutionPolicy']['mode'])
        # Uncomment the next line to see the full object
        # print(coll[0])
    else:
        print("Collection", coll_name, "not found :(")
else:
    print("Database", db_name, "not found :(")


Database ecommerce found
Collection customers found:
 - Indexing policy - Mode: consistent
 - Indexing policy - Mode: True
 - Conflict resolution policy mode: LastWriterWins


Alternatively to the QueryContainers method, you can craft manually a collection link with the database and collection names, and use the method ReadCollection:

In [69]:
coll_link='dbs/{0}/colls/{1}/'.format(db_name, coll_name)
print("Collection link:", coll_link)
collection = client.ReadContainer(coll_link)
print(collection)

dbs/ecommerce/colls/customers/
{'id': 'customers', 'indexingPolicy': {'indexingMode': 'consistent', 'automatic': True, 'includedPaths': [{'path': '/*', 'indexes': [{'kind': 'Range', 'dataType': 'Number', 'precision': -1}, {'kind': 'Range', 'dataType': 'String', 'precision': -1}]}], 'excludedPaths': [{'path': '/"_etag"/?'}]}, 'partitionKey': {'paths': ['/id'], 'kind': 'Hash', 'systemKey': False}, 'conflictResolutionPolicy': {'mode': 'LastWriterWins', 'conflictResolutionPath': '/_ts', 'conflictResolutionProcedure': ''}, 'geospatialConfig': {'type': 'Geography'}, '_rid': 'xV1rAOWWurk=', '_ts': 1556489716, '_self': 'dbs/xV1rAA==/colls/xV1rAOWWurk=/', '_etag': '"00009107-0000-0d00-0000-5cc625f40000"', '_docs': 'docs/', '_sprocs': 'sprocs/', '_triggers': 'triggers/', '_udfs': 'udfs/', '_conflicts': 'conflicts/'}


## Deleting an existing database or collection

If you want to play with the different options to create a collection, you can modify most of its attributes, with the notable exception of the partition key. If you need to try the commands above with different options you might have to delete the collection first.

In [45]:
# Delete only a specific collection
coll_query = "select * from r where r.id = '{0}'".format(coll_name)
coll = list(client.QueryContainers(db_link, coll_query))
if coll:
    coll_link = coll[0]['_self']
    client.DeleteContainer(coll_link)
    print("Collection", coll_name, "has been deleted")
else:
    print("Collection", coll_name, "could not be found")

Collection customers has been deleted


In [25]:
# If the DB exists, delete it (including any collections)
db_query = "select * from r where r.id = '{0}'".format(db_name)
db = list(client.QueryDatabases(db_query))
if db:
    db_link = db[0]['_self']
    client.DeleteDatabase(db_link)
    print("Database", db_name, "and all its collections have been deleted")
else:
    print("Database", db_name, "could not be found")

Database ecommerce and all its collections have been deleted


## Updating the RUs of an existing collection

You might want to update the performance (throughput) of your collection under certain circumstances, for example occasionally to perform a performance-intensive operation such as a data import or export, or just because you need more performance on your database on a consistent basis.

To read the current throughput of an existing collection you can use the QueryOffers method, and update it with the ReplaceOffers method.

In [65]:
coll_query = "select * from r where r.id = '{0}'".format(coll_name)
coll = list(client.QueryContainers(db_link, coll_query))
if coll:
    # We found our collection, get the link and find the offers
    coll_link = coll[0]['_self']
    offer = list(client.QueryOffers('SELECT * FROM c WHERE c.resource = \'{0}\''.format(coll_link)))[0]
    print("Collection", coll_name, "found provisioned with", str(offer['content']['offerThroughput']), "RU/s")
else:
    print("Collection", coll_name, "not found")

Collection customers found provisioned with 500 RU/s


In [64]:
new_offer = offer
new_offer['content']['offerThroughput'] += 50
throughput = new_offer['content']['offerThroughput']
if (throughput >= 400) and (throughput <= 100000) and ((throughput % 100) == 0): 
    offer = client.ReplaceOffer(offer['_self'], new_offer)
else:
    print(throughput, "is not a valid throughput for Cosmos DB")

## Create documents

You can use the Python SDK to create new documents in our collection. For this we will use some sample documents, that are used as well in the [EDX course for Cosmos DB](https://courses.edx.org/courses/course-v1:Microsoft+DAT237x+2T2017/course/). In that course these documents are used to illustrate the .NET API, we will use them here for the Python SDK.

The function insert_item supports a "pretrigger" argument, please ignore it for the time being (we will come back to it later in this tutorial).

In [31]:
import json

# Single add
def insert_item(client, coll_link, item, pretrigger=None):
    options={}
    if pretrigger:
        options['preTriggerInclude'] = pretrigger
    client.CreateItem(coll_link, item, options)

# Add all documents in a specific folder
def insert_items_batch(client, coll_link, data_dir):
    jsonpath = os.path.join(os.getcwd(), data_dir)
    counter=0
    for file in os.listdir(jsonpath):
        if file[-5:] == ".json":
            #print("Found json file", file)
            f=open(os.path.join(jsonpath, file), 'r')
            item_json = f.read()
            item = json.loads(item_json)
            insert_item(client, coll_link, item)
            counter += 1
        else:
            print("Non-JSON file found:", file)
    print(counter, "documents added")

insert_items_batch(client, coll_link, 'data')

50 documents added


## Querying documents

Now that we have some data in the collection, we can start running some queries. Let us start with something simple. As you can see, the send_query function supports defining query options, we will see those in a minute. See the reference documentation for the [QueryItems](https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python#queryitems-database-or-container-link--query--options-none--partition-key-none-) method for more information.

In the example below, for output clarity the send_query function has an optional toggle that can be used to hide the actual results, and print only the number of matching documents, as well as the RU/s consumed by this particular query. Note how the consumed RU's are obtained by looking at a specific field of the client object, that contains the consumed RU's for the last operation.

In [52]:
# Auxiliary function that we will use to send a query to a collection
def send_query(client, coll_link, query, options=None, show_results=True):
    docs = list(client.QueryItems(coll_link, query, options))
    for doc in docs:
        if show_results:
            print(json.dumps(doc, indent=4, sort_keys=True))
    print(len(docs), 'items found,', client.last_response_headers['x-ms-request-charge'], 'RU/s consumed')

In [54]:
# Simple query
customer_id='09d2bb28e9c54bc581492d542789f2ad'
query = 'SELECT * FROM customers WHERE customers.id=\'{0}\''.format(customer_id)
send_query(client, coll_link, query, show_results=False)

1 items found, 2.91 RU/s consumed


As you might have seen in the code for send_query, you can send some options along with the query, that will modify the way in which the query is executed. For example, we can enable cross-partition queries (to be able to select all records) and set the maximum item count to 2:

In [55]:
# Simple query with options
query = 'SELECT * FROM customers'
options = {} 
options['enableCrossPartitionQuery'] = True
options['maxItemCount'] = 2
send_query(client, coll_link, query, options, show_results=False)

50 items found, 2.32 RU/s consumed


More complex queries are supported using the SQL dialect of Cosmos DB:

In [56]:
# Query showing projection and filtering
query='''SELECT {
     "full-name": customers.name,
     "contact-details": {
        "phone": customers.phone,
        "address": customers.address
    },
     "employment": {
        "employer": customers.company,
        "work-email": customers.email
     }
} AS person
FROM customers
WHERE customers.source = "retail-location"'''
options = {} 
options['enableCrossPartitionQuery'] = True
send_query(client, coll_link, query, options, show_results=False)

5 items found, 3.16 RU/s consumed


## Triggers

Triggers are activites that can be executed before ("Pre" triggers) or after ("Post" triggers) data being written or updated in Cosmos DB. Triggers are written in JavaScript, which makes working with JSON very easy. In this repository we have a trigger that adds a 'department' property to a document before adding it to the collection, in case that property did not exist.

In [12]:
import os
trigger_filename = os.path.join(os.getcwd(), 'triggers', 'validateDepartmentExists.js')
f=open(trigger_filename, 'r')
trigger_code = f.read()
print(trigger_code)

function validateDepartmentExists() {
    var context = getContext();
    var request = context.getRequest();
    var newDocument = request.getBody();

    if (!newDocument.department) {
        newDocument["department"] = "General"
    }

    request.setBody(newDocument);
}


In [58]:
import azure.cosmos.documents as documents

# Create new trigger from file
def create_trigger(client, coll_link, filename):
    with open(filename) as file:
        file_contents = file.read()
    trigger_name = os.path.splitext(os.path.basename(filename))[0]
    trigger_definition = {
        'id': trigger_name,
        'serverScript': file_contents,
        'triggerType': documents.TriggerType.Pre,
        'triggerOperation': documents.TriggerOperation.All
    }
    trigger = client.CreateTrigger(coll_link, trigger_definition)

# List existing defined triggers in the collection
def query_triggers(client, coll_link):
    trigger_query = 'select * from r'
    triggers = list(client.QueryTriggers(coll_link, trigger_query))
    for trigger in triggers:
        print(json.dumps(trigger, indent=4, sort_keys=True))

    
trigger_filename = os.path.join(os.getcwd(), 'triggers', 'validateDepartmentExists.js')
create_trigger(client, coll_link, trigger_filename)
query_triggers(client, coll_link)

{
    "_etag": "\"3b001ae8-0000-0d00-0000-5cc6cd050000\"",
    "_rid": "PclqAM-oEm0BAAAAAAAAcA==",
    "_self": "dbs/PclqAA==/colls/PclqAM-oEm0=/triggers/PclqAM-oEm0BAAAAAAAAcA==/",
    "_ts": 1556532485,
    "body": "function validateDepartmentExists() {\n    var context = getContext();\n    var request = context.getRequest();\n    var newDocument = request.getBody();\n\n    if (!newDocument.department) {\n        newDocument[\"department\"] = \"General\"\n    }\n\n    request.setBody(newDocument);\n}",
    "id": "validateDepartmentExists",
    "triggerOperation": "all",
    "triggerType": "pre"
}


We can now add a new document without the 'department' property, and verify that the property was added by the Trigger. For that we will use a helper function that generates a customer with a random ID:

In [63]:
import random, string

def get_random_item():
    random_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=16))
    itemToCreate = {
        'firstName': 'Sample',
        'lastName': 'Person',
        'id': random_id
    }
    return itemToCreate

random_item = get_random_item()
print('We will send an item without the \'department\' property:')
print(random_item)
# Note that we do specify the ID of the Pre trigger to execute
insert_item(client, coll_link, random_item, pretrigger='validateDepartmentExists')
print("And now the item actually created should have the \'department\' property, added by the trigger:")
query = 'SELECT * FROM customers WHERE customers.id = \'{0}\''.format(random_item['id'])
send_query(client, coll_link, query)


We will send an item without the 'department' property:
{'firstName': 'Sample', 'lastName': 'Person', 'id': 'i4q2jjd4s0jpgqlf'}
And now the item actually created should have the 'department' property, added by the trigger:
{
    "_attachments": "attachments/",
    "_etag": "\"3b0034e8-0000-0d00-0000-5cc6ceae0000\"",
    "_rid": "PclqAM-oEm21hB4AAAAAAA==",
    "_self": "dbs/PclqAA==/colls/PclqAM-oEm0=/docs/PclqAM-oEm21hB4AAAAAAA==/",
    "_ts": 1556532910,
    "department": "General",
    "firstName": "Sample",
    "id": "i4q2jjd4s0jpgqlf",
    "lastName": "Person"
}
1 items found, 2.9 RU/s consumed


## Cleanup

In order to save costs you can delete the resource group that was created at the beginning of this exercise via the Azure CLI (or any other method). Please be careful with this command, since it will delete the resource group as well as every resource inside, including the Cosmos DB account and all databases and collections.

In [2]:
!az group delete -n $rg -y --no-wait